# Udacity Optimizing ML Pipeline - Bank Marketing Classification

The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution.
The classification goal is to predict if the client will subscribe a term deposit (variable y).

* Bank marketing dataset
    * https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
    * https://www.kaggle.com/c/bank-marketing-uci

### Create or Get workspace

In [1]:
from azureml.core import Workspace, Experiment

workspace = "MLworkspace"
workspace = "quick-starts-ws-133008"

ws = Workspace.get(name=workspace)
exp = Experiment(workspace=ws, name="udacity-Hypetune-BankMarketing-classification")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

Workspace name: MLworkspace
Azure region: eastus
Subscription id: 5831e312-31c6-42ba-b4a9-5c780e5ea1da
Resource group: MLAzureGroup


### Create compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

clustername = 'GPU-cluster'
is_new_cluster = False
try:
    gpu_cluster = ComputeTarget(workspace = ws,name= clustername)
    print("Find the existing cluster")
except ComputeTargetException:
    print("Cluster not find - Creating cluster")
    is_new_cluster = True
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    gpu_cluster = ComputeTarget.create(ws, clustername, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

Find the existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Setup Environment

In [3]:
from azureml.core import ScriptRunConfig, Environment
env = Environment.get(workspace=ws, name="AzureML-Tutorial")

## Part 1-  Use HyperDrive to automatically find optimal parameters

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive import choice

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C":choice(1,3),
    "--max_iter":choice(100,1000)
})


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=gpu_cluster,
                      environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config= est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-Hypetune-BankMarketing-classification,HD_cd8deaf6-33b3-40da-89f8-e8635b4c6175,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
hyperdrive_run.get_status()

'Running'

In [7]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

### Retrieve the best run

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-Hypetune-BankMarketing-classification,HD_cd8deaf6-33b3-40da-89f8-e8635b4c6175_3,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
# The model was saved in the train scripts
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_48da0cfee50dd391a2bb872ab27175a5f9729bd001ea53285de61fffd0867ba5_p.txt',
 'azureml-logs/65_job_prep-tvmps_48da0cfee50dd391a2bb872ab27175a5f9729bd001ea53285de61fffd0867ba5_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_48da0cfee50dd391a2bb872ab27175a5f9729bd001ea53285de61fffd0867ba5_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/109_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/engine_spans_1b1b5e61-2836-4f9e-a1dd-4009678b8bb2.jsonl',
 'logs/azureml/dataprep/python_span_1b1b5e61-2836-4f9e-a1dd-4009678b8bb2.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/hyper_model.joblib',
 'outputs/model.pkl']

### Register the best model

In [10]:
model = best_run.register_model(model_name='hyper_model', model_path=os.path.join('outputs', 'model.pkl'))
print(model.name, model.version, sep = '\t')

hyper_model	2


## Part 2 -  AutoML Run

* We want to compare the previous results with Auto ML

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

### Train Test Split

In [12]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.25,stratify=y)


In [13]:
import pandas as pd
train_data = pd.concat([x_train,y_train],axis =1)
train_data.to_csv('./training/train_data.csv')

### Training data

In [14]:
#Because auto ml in azure only accept datasets NOT dataframe

from azureml.core.dataset import Dataset
ds_tr = ws.get_default_datastore()
ds_tr.upload(src_dir='./training', target_path='training_data', overwrite=True, show_progress=True)
training_data =TabularDatasetFactory.from_delimited_files(path=ds_tr.path('training_data/train_data.csv'))

Uploading an estimated of 2 files
Uploading ./training/hyper_model.joblib
Uploaded ./training/hyper_model.joblib, 1 files out of an estimated total of 2
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [15]:
training_data

{
  "source": [
    "('workspaceblobstore', 'training_data/train_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

### Auto ML configuration

In [16]:
%%time
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric="accuracy",
    compute_target = gpu_cluster,
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5)

CPU times: user 61.9 ms, sys: 31.5 ms, total: 93.4 ms
Wall time: 548 ms


### Auto ML run

In [17]:
%%time
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on GPU-cluster with default configuration
Running on remote compute: GPU-cluster
Parent Run ID: AutoML_717b3f0a-423e-46cd-8df1-88a09f00ff98

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+------------

In [19]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [22]:
# Retrieve and save your best automl model.
automl_best_run, fitted_model = automl_run.get_output()
print(automl_best_run)
print(fitted_model)

AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

### Delete compute ressources

In [ ]:
if is_new_cluster :
    gpu_cluster.delete()